In [ ]:
import pyspark.sql.functions as fn

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load from Parquet

In [ ]:
df = spark.read.parquet('dataset/THB_cards.parquet')

In [ ]:
df.createOrReplaceTempView("cards")

In [ ]:
df.columns

## Groups 

### By color identity

In [ ]:
df.groupby('colorIdentity').count().show()

### From Models, show encoded types

In [ ]:
from pyspark.ml.feature import StringIndexer, StringIndexerModel

In [ ]:
indexer = StringIndexer.load("models/pyspark/stringindexer_str_types")
model = StringIndexerModel.load("models/pyspark/stringindexer_model_str_types")

In [ ]:
model.labels

In [ ]:
df.agg(fn.min(df.encodedTypes),fn.max(df.encodedTypes)).show()

### By color identity and encoded type

First, let's fetch the distinct color identities

In [ ]:
pd_colorIdentities  = df.select("colorIdentity").alias("Color_Identity").distinct().sort("colorIdentity").toPandas()

In [ ]:
pd_colorIdentities

In [ ]:
pd_encodedTypes = pd.DataFrame(np.arange(9), columns=['Encoded_Types'])

In [ ]:
pd_encodedTypes

Create a cartesian product of the color identities and the encoded types.

In [ ]:
pd_colorIdentities['key'] = 0
pd_encodedTypes['key'] = 0

In [ ]:
pd_cartesian = pd_colorIdentities.merge(pd_encodedTypes, how='outer').drop(columns=['key'])

In [ ]:
pd_cartesian = pd_cartesian.rename(columns={'colorIdentity':'Color_Identity'})

In [ ]:
pd_cartesian

Now, let's create a list which will hold the following: ((color identity, encoded type), 1).

In [ ]:
map1 = df.select(["colorIdentity", "encodedTypes"]).rdd.map(
    lambda x: ((x[0], x[1]), 1)
)

map1.take(10)

Finally, let's compute the sum of the number of a found encoded types per color identity.

In [ ]:
map2 = map1.reduceByKey(lambda a, b: a + b).sortByKey()

In [ ]:
map2.take(5)

For convience, let's break down the (color identity, encoded type) key into a list of: (color identity, encoded type, sum). 

In [ ]:
map3 = map2.map(lambda x: (x[0][0], x[0][1], x[1]))

In [ ]:
pd_tab = map3.toDF().toPandas()

In [ ]:
pd_tab.columns = ['Color_Identity', 'Encoded_Types', 'Total']

In [ ]:
pd_tab.head(5)

Based on the color identity and encoded types, assign a key that corresponds to the cartesian product.

In [ ]:
def assign_index(row):
    filter1 = pd_cartesian['Color_Identity'] ==  row['Color_Identity']
    filter2 = pd_cartesian['Encoded_Types'] == row['Encoded_Types']
    
    idx = pd_cartesian.index[filter1 & filter2].tolist()[0]

    return idx

In [ ]:
pd_tab['Cart_Index'] = pd_tab.apply (lambda row: assign_index(row), axis=1)

In [ ]:
pd_tab.head(5)

Find the missing indexes from the cartesian product, and fill-in with "total = 0".

In [ ]:
missing_cart_indexes = set(pd_cartesian.index.tolist()) - set(pd_tab.Cart_Index.tolist())

In [ ]:
for index in missing_cart_indexes:
    row = pd_cartesian.iloc[index]

    new_df = row.copy()
    new_df['Total'] = 0
    new_df['Cart_Index'] = index
    
    pd_tab = pd_tab.append(new_df)

In [ ]:
group_result = pd_tab.groupby('Encoded_Types')

In [ ]:
group_result.groups

In [ ]:
for group in group_result.groups.items():
    pass

In [ ]:
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(6, 6))

sns.set_color_codes("pastel")
sns.barplot(x="Total", y="Color_Identity", data=pd_tab, label="Total", orient='h', color="b", )

sns.despine(left=True, bottom=True)

### Make an overall histogram from encoded types. 

In [ ]:
hist = df.select("encodedTypes").rdd.flatMap(lambda x: x).histogram(9)

In [ ]:
hist

In [ ]:
pd_hist = pd.DataFrame(data=list(zip(*hist)), columns=['bin', 'freq'])

In [ ]:
pd_hist['perc'] = pd_hist['freq'] / pd_hist['freq'].sum()

In [ ]:
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(6, 6))

sns.set_color_codes("pastel")
sns.barplot(x="perc", y="bin", data=pd_hist, label="Total", orient='h', color="b")

ax.set(xlim=(0, 0.4), xlabel="", ylabel="")
# ax.set(xlim=(0, 0.5), xlabel="%", ylabel="Creature type")
ax.set(yticklabels=model.labels)

sns.despine(left=True, bottom=True)

## SQL / FN

In [ ]:
df.agg(*[fn.max(c).alias('max ' + c) for c in ['power', 'toughness']]).show()

In [ ]:
spark.sql("SELECT * FROM cards WHERE power == 10").show()

In [ ]:
spark.sql("""
    SELECT * FROM cards
    ORDER BY power DESC, toughness DESC
    LIMIT 5
""").show()